In [1]:
# Import required libraries
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import WebDriverException, TimeoutException, NoSuchElementException, StaleElementReferenceException, ElementClickInterceptedException
from datetime import datetime
import time
import os
import wget
import json
import csv

In [2]:
# Set up ChromeDriver
service = Service('E:/Programs/Chrome Driver/chromedriver.exe')
driver = webdriver.Chrome(service=service)

In [3]:
import pickle

# Define the file path to load the data
file_path = 'hrefs_with_categories.pkl'

# Load the data using pickle
with open(file_path, 'rb') as file:
    hrefs_with_categories = pickle.load(file)

In [4]:
for href, category in hrefs_with_categories:
    if category.lower() == "sports":
        driver.get(href)
        break 

In [5]:
# getting primary page news
# Initialize the list to store hrefs
primary_hrefs = []

# Define the XPath for the elements to extract the href
xpath = ('//*[contains(concat( " ", @class, " " ), concat( " ", "card_1", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "O1LaH", " " ))] | '
         '//*[contains(concat( " ", @class, " " ), concat( " ", "card_2", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "O1LaH", " " ))] | '
         '//*[contains(concat( " ", @class, " " ), concat( " ", "card_3", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "WavNE", " " ))] | '
         '//*[contains(concat( " ", @class, " " ), concat( " ", "card_13", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "O1LaH", " " ))] | '
         '//*[contains(concat( " ", @class, " " ), concat( " ", "card_4", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "WavNE", " " ))] | '
         '//*[contains(concat( " ", @class, " " ), concat( " ", "card_14", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "undefined", " " ))] | '
         '//*[contains(concat( " ", @class, " " ), concat( " ", "card_6", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "WavNE", " " ))] | '
         '//*[contains(concat( " ", @class, " " ), concat( " ", "card_5", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "O1LaH", " " ))]')

# Wait for the elements to be present
try:
    elements = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, xpath))
    )

    for element in elements:
        try:
            # Find the parent <a> tag
            parent_a_tag = element.find_element(By.XPATH, "./ancestor::a")

            # Extract the href attribute from the parent <a> tag
            href = parent_a_tag.get_attribute('href')
            
            # Append the href to the primary_hrefs list if it exists
            if href:
                primary_hrefs.append(href)

        except NoSuchElementException as e:
            print(f"Error retrieving href: {e}")
            continue

except NoSuchElementException as e:
    print(f"Error locating elements: {e}")

# print(f"Collected hrefs: {primary_hrefs}")
print(f"The number of collected hrefs : {len(primary_hrefs)}")

The number of collected hrefs : 130


In [6]:
# Initialize the list to store all sports hrefs
all_sports_href = []

# Simplified XPath to test for potential issues
sports_xpath = ('//div[count(preceding-sibling::*) + 1 = 10]//*[contains(@class, "o1wRt")] | '
                '//div[count(preceding-sibling::*) + 1 = 12]//*[contains(@class, "o1wRt")] | '
                '//div[count(preceding-sibling::*) + 1 = 14]//*[contains(@class, "o1wRt")] | '
                '//*[contains(@class, "leaderboard_placeHolder")]//div//*[contains(@class, "o1wRt")] | '
                '//div[count(preceding-sibling::*) + 1 = 18]//*[contains(@class, "o1wRt")] | '
                '//div[count(preceding-sibling::*) + 1 = 20]//*[contains(@class, "o1wRt")] | '
                '//div[count(preceding-sibling::*) + 1 = 25]//*[contains(@class, "o1wRt")] | '
                '//*[contains(@class, "baeqq")]//*[contains(@class, "o1wRt")] | '
                '//div[count(preceding-sibling::*) + 1 = 27]//*[contains(@class, "o1wRt")]')

try:
    # Wait for the elements to be present
    sports_elements = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, sports_xpath))
    )

    for element in sports_elements:
        try:
            # Find the parent <a> tag
            parent_a_tag = element.find_element(By.XPATH, "./ancestor::a")

            # Extract the href attribute from the parent <a> tag
            href = parent_a_tag.get_attribute('href')
            
            # Append the href to the all_sports_href list if it exists
            if href:
                all_sports_href.append(href)

        except NoSuchElementException as e:
            print(f"Error retrieving href for an element: {e}")
            continue

except (NoSuchElementException, InvalidSelectorException) as e:
    print(f"Error locating sports elements: {e}")

# Filter out any hrefs that end with '/sports'
all_sports_href = [href for href in all_sports_href if not href.endswith('/sports')]

# Print out the filtered all_sports_href list
print(f"Collected sports hrefs: {all_sports_href}")

Collected sports hrefs: ['https://timesofindia.indiatimes.com/sports/cricket', 'https://timesofindia.indiatimes.com/sports/football', 'https://timesofindia.indiatimes.com/sports/tennis', 'https://timesofindia.indiatimes.com/sports/wwe', 'https://timesofindia.indiatimes.com/sports/nfl', 'https://timesofindia.indiatimes.com/sports/hockey', 'https://timesofindia.indiatimes.com/sports/boxing', 'https://timesofindia.indiatimes.com/sports/golf']


In [7]:
# Loop through each href in the all_sports_href list
for href in all_sports_href:
    try:
        # Direct the driver to the first page of each link
        driver.get(href)
        
        # Wait for the page to load completely
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.TAG_NAME, "body")) 
        )

        # Print the current URL to verify navigation
        print(f"Navigated to: {driver.current_url}")
        
        # Check if the current URL is the cricket page
        if "cricket" in driver.current_url:
            cricket_hrefs = []

            # Define the XPath expressions for scraping
            cricket_xpath = (
                '//*[contains(concat(" ", @class, " "), concat(" ", "linktype2", " "))] | '
                '//*[contains(concat(" ", @class, " "), concat(" ", "col_m_3", " "))]//figcaption | '
                '//*[contains(concat(" ", @class, " "), concat(" ", "linktype1", " "))]//span | '
                '//*[contains(concat(" ", @class, " "), concat(" ", "sNF1c", " "))] | '
                '//h5 | '
                '//*[contains(concat(" ", @class, " "), concat(" ", "BC4XY", " "))]//a'
            )

            cricket_elements = driver.find_elements(By.XPATH, cricket_xpath)

            for element in cricket_elements:
                try:
                    parent_a_tag = element.find_element(By.XPATH, "./ancestor::a")
                    href = parent_a_tag.get_attribute('href')
                    if href:
                        cricket_hrefs.append(href)
                except NoSuchElementException:
                    continue

            # Loop through pages 2 to 20 for the cricket section
            for page_number in range(2, 21):
                page_url = f"https://timesofindia.indiatimes.com/sports/cricket/{page_number}"
                driver.get(page_url)

                # Wait for the page to load
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.TAG_NAME, "body"))
                )

                # Print the current URL to verify navigation
                print(f"Navigated to page: {driver.current_url}")

                additional_cricket_xpath = '//*[contains(concat(" ", @class, " "), concat(" ", "Jnjss", " "))]'
                additional_cricket_elements = driver.find_elements(By.XPATH, additional_cricket_xpath)

                for element in additional_cricket_elements:
                    try:
                        parent_a_tag = element.find_element(By.XPATH, "./ancestor::a")
                        href = parent_a_tag.get_attribute('href')
                        if href:
                            cricket_hrefs.append(href)
                    except NoSuchElementException:
                        continue

            print(f"Number of collected cricket hrefs: {len(cricket_hrefs)}")
            
        # Check if the current URL is the WWE page
        elif "wwe" in driver.current_url:
            while True:
                try:
                    button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "O92W7", " "))]'))
                    )
                    button.click()
                    time.sleep(1)  # Adjust sleep duration if necessary
                except (NoSuchElementException, TimeoutException):
                    print("No more buttons to click in WWE, proceeding with href extraction.")
                    break
                    
            wwe_hrefs = []

            wwe_xpath = (
                '//figcaption | '
                '//h5 | '
                '//*[contains(concat(" ", @class, " "), concat(" ", "null", " "))]//span'
            )

            wwe_elements = driver.find_elements(By.XPATH, wwe_xpath)

            for element in wwe_elements:
                try:
                    parent_a_tag = element.find_element(By.XPATH, "./ancestor::a")
                    href = parent_a_tag.get_attribute('href')
                    if href:
                        wwe_hrefs.append(href)
                except NoSuchElementException:
                    continue
            
            print(f"Collected number of WWE hrefs: {len(wwe_hrefs)}")
            
        elif "golf" in driver.current_url:
            golf_hrefs = []
        
            for i in range(1, 52):
                golf_page_url = f"https://timesofindia.indiatimes.com/sports/golf/{i}"
                
                try:
                    driver.get(golf_page_url)
                    
                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.TAG_NAME, "body"))
                    )
                    
                    print(f"Navigated to: {driver.current_url}")
        
                    golf_xpath = (
                        '//*[contains(concat(" ", @class, " "), concat(" ", "js-main-news-list", " "))]'
                        '//*[contains(concat(" ", @class, " "), concat(" ", "clearfix", " "))]//li//a'
                    )

        
                    golf_elements = driver.find_elements(By.XPATH, golf_xpath)
        
                    for element in golf_elements:
                        try:
                            href = element.get_attribute('href')
                            if href:
                                golf_hrefs.append(href)
                        except NoSuchElementException:
                            continue
                    
                except Exception as e:
                    print(f"Error navigating to {golf_page_url}: {e}")
                    continue
        
            print(f"Collected Golf hrefs: {len(golf_hrefs)}")
            
        elif "boxing" in driver.current_url:
            boxing_hrefs = []
            
            boxing_xpath = (
                '//*[@id="c_sport_wdt_1"]//*[contains(concat(" ", @class, " "), concat(" ", "w_tle", " "))]//a'
            )

            boxing_elements = driver.find_elements(By.XPATH, boxing_xpath)

            for element in boxing_elements:
                try:
                    href = element.get_attribute('href')
                    if href:
                        boxing_hrefs.append(href)
                except NoSuchElementException:
                    continue

            print(f"Collected Boxing hrefs: {len(boxing_hrefs)}")
            
        elif "hockey" in driver.current_url:
            hockey_hrefs = []

            # Loop through the main Hockey pages
            for i in range(1, 2):  # Adjust the range as necessary
                hockey_page_url = f"https://timesofindia.indiatimes.com/sports/hockey/{i}"
                try:
                    # Direct the driver to the hockey page
                    driver.get(hockey_page_url)
            
                    # Wait for the page to load completely
                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.TAG_NAME, "body"))  # Ensure page load
                    )
            
                    # Print the current URL to verify navigation
                    print(f"Navigated to: {driver.current_url}")
            
                    # Define the XPath expressions for scraping Hockey hrefs
                    hockey_xpath = (
                        '//*[@id="c_sport_wdt_1"]//*[contains(concat(" ", @class, " "), concat(" ", "w_tle", " "))]//a'
                    )
            
                    # Find all elements using the specified XPath
                    hockey_elements = driver.find_elements(By.XPATH, hockey_xpath)
            
                    # Extract hrefs from each element found
                    for element in hockey_elements:
                        try:
                            # Extract href from the parent <a> tag
                            href = element.get_attribute('href')
                            if href:
                                hockey_hrefs.append(href)
                        except NoSuchElementException:
                            continue
            
                    print(f"Collected Hockey hrefs from page {i}: {len(hockey_elements)}")
            
                    # Click the button to load additional content if available
                    try:
                        button = WebDriverWait(driver, 10).until(
                            EC.element_to_be_clickable((By.XPATH, '//*[contains(concat(" ", @class, " "), concat(" ", "sweep-button", " "))]//a'))
                        )
                        clicked_url = button.get_attribute('href') 
                        button.click()
                        time.sleep(1)  # Wait for the new content to load
                    except (NoSuchElementException, TimeoutException):
                        print("No button to click for more hockey content or the button is not clickable.")
            
                except Exception as e:
                    print(f"Error navigating to {hockey_page_url}: {e}")
            
            # Navigate to additional pages (/1 and /2) after clicking the button
            for page_number in range(1, 3):  # Adjust the range as necessary
                new_hockey_page_url = f"{clicked_url}/{page_number}"
                driver.get(new_hockey_page_url)
                
                # Wait for the new page to load
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.TAG_NAME, "body"))  # Ensure page load
                )
                
                # Print the current URL to verify navigation
                print(f"Navigated to additional page: {driver.current_url}")
            
                # Find all elements using the same XPath for additional pages
                hockey_elements = driver.find_elements(By.XPATH, hockey_xpath)
            
                # Extract hrefs from each element found on the additional page
                for element in hockey_elements:
                    try:
                        # Extract href from the parent <a> tag
                        href = element.get_attribute('href')
                        if href:
                            hockey_hrefs.append(href)
                    except NoSuchElementException:
                        continue
            
            # Print the total number of collected Hockey hrefs
            print(f"Total collected Hockey hrefs: {len(hockey_hrefs)}")

        elif "nfl" in driver.current_url:
            nfl_hrefs = []
            
            # Get the current URL
            current_url = driver.current_url
            
            # Loop through pages from 1 to 34
            for page_number in range(1, 35):  # 1 to 34
                nfl_page_url = f"{current_url}/{page_number}"  # Construct the page URL
                try:
                    # Direct the driver to the NFL page
                    driver.get(nfl_page_url)
        
                    # Optionally, wait for the page to load completely
                    WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.TAG_NAME, "body"))  # Ensure page load
                    )
        
                    # Print the current URL to verify navigation
                    print(f"Navigated to: {driver.current_url}")
        
                    # Define the XPath expressions for scraping NFL hrefs
                    nfl_xpath = (
                        '//*[@id="c_sport_wdt_1"]//*[contains(concat(" ", @class, " "), concat(" ", "w_tle", " "))]//a'
                    )
        
                    # Find all elements using the specified XPath
                    nfl_elements = driver.find_elements(By.XPATH, nfl_xpath)
        
                    # Extract hrefs from each element found
                    for element in nfl_elements:
                        try:
                            # Extract href from the parent <a> tag
                            href = element.get_attribute('href')
                            if href:
                                nfl_hrefs.append(href)
                        except NoSuchElementException:
                            continue
        
                except Exception as e:
                    print(f"Error navigating to {nfl_page_url}: {e}")
        
            # Print collected NFL hrefs
            print(f"Collected NFL hrefs: {len(nfl_hrefs)}")

        elif "tennis" in driver.current_url:
            tennis_hrefs = []
            tennis_categories = driver.find_elements(By.XPATH, '//h2[contains(@class, "peJf5")]//a[contains(@class, "HjSby")]')
            tennis_categories_href = []
            
            # Extract hrefs from the <a> tags inside the <h2> elements
            for category in tennis_categories:
                try:
                    href = category.get_attribute('href')
                    if href:
                        tennis_categories_href.append(href)
                except NoSuchElementException:
                    continue
        
            # Iterate through the hrefs in tennis_categories_href
            for href in tennis_categories_href:
                for i in range(1, 34):  # Loop through the pages (up to 33)
                    driver.get(f"{href}/{i}")
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
        
                    # Find elements in the js-main-news-list with the parent <a> tag whose href needs to be collected
                    tennis_elements = driver.find_elements(By.XPATH, '//*[contains(concat(" ", @class, " "), concat(" ", "js-main-news-list", " "))]//li//a')
                    
                    # Extract the hrefs from these <a> elements
                    for element in tennis_elements:
                        href = element.get_attribute('href')
                        if href:
                            tennis_hrefs.append(href)
        
            print(f"Collected Tennis hrefs: {len(tennis_hrefs)}")

        elif "football" in driver.current_url:
            # List to hold football hrefs and category hrefs
            football_hrefs = []
            football_categories_href = []
            
            # Click the 'Load More STORIES' button until it's no longer available
            while True:
                try:
                    load_more_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, '//button[contains(@class, "O92W7")]'))
                    )
                    load_more_button.click()
                    print("Clicked 'Load More STORIES' button")
            
                    # Wait for new content to load
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
                    time.sleep(1)  # Optional: short wait after clicking
            
                except TimeoutException:
                    print("Load More button not found or clickable. Exiting.")
                    break
                except StaleElementReferenceException:
                    print("StaleElementReferenceException encountered. Retrying to find the button...")
                    continue  # Retry the loop to find and click again
                except (NoSuchElementException, ElementClickInterceptedException):
                    print("No more 'Load More STORIES' button to click.")
                    break

            
            # Once all the content is loaded, collect hrefs from the specified elements
            football_elements = driver.find_elements(By.XPATH, 
                '//*[contains(concat( " ", @class, " " ), concat( " ", "col_l_4", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "hoid1", " " ))]//figcaption | '
                '//*[contains(concat( " ", @class, " " ), concat( " ", "null", " " ))]//span'
            )
            # Extract hrefs from the parent <a> tags of these elements
            for element in football_elements:
                try:
                    parent_a_tag = element.find_element(By.XPATH, './ancestor::a')
                    href = parent_a_tag.get_attribute('href')
                    if href:
                        football_hrefs.append(href)
                except NoSuchElementException:
                    continue
            
            # Collect category hrefs from the elements matching the second XPath
            football_categories = driver.find_elements(By.XPATH, '//*[contains(concat(" ", @class, " "), concat(" ", "xhx_z", " "))]')
            
            # Extract hrefs from the parent <a> tags of these category elements
            for category in football_categories:
                try:
                    parent_a_tag = category.find_element(By.XPATH, './ancestor::a')
                    href = parent_a_tag.get_attribute('href')
                    if href:
                        football_categories_href.append(href)
                except NoSuchElementException:
                    continue

            print(f"Collected Football Category hrefs: {len(football_categories_href)}")

            # Iterate through each category href
            for category_href in football_categories_href:
                for page_number in range(1, 61):  # Loop through pages 1 to 60
                    # Construct the URL for each page
                    page_url = f"{category_href}/{page_number}"
                    driver.get(page_url)
            
                    # Wait for the page to load
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
            
                    try:
                        # Get hrefs from the specified elements
                        football_elements = driver.find_elements(By.XPATH, 
                            '//*[(@id = "c_sport_wdt_1")]//*[contains(concat( " ", @class, " " ), concat( " ", "w_tle", " " ))]//a'
                        )
            
                        # Extract hrefs and store them in the list
                        if football_elements:  # Check if any elements were found
                            for element in football_elements:
                                href = element.get_attribute('href')
                                if href:
                                    football_hrefs.append(href)
                    except NoSuchElementException:
                        # If no elements are found, continue to the next page indentation
                        continue
                    except Exception as e:
                        # Optionally, log or print the error
                        print(f"Error accessing {page_url}: {e}")
                        continue
            
            # Print the number of collected hrefs
            print(f"Collected Football hrefs: {len(football_hrefs)}")
            
            # Optional: Print the first few hrefs to check the results
            # print(football_hrefs[:10])
        
        
    except Exception as e:
        print(f"Error navigating to {href}: {e}")


Navigated to: https://timesofindia.indiatimes.com/sports/cricket
Navigated to page: https://timesofindia.indiatimes.com/sports/cricket/2
Navigated to page: https://timesofindia.indiatimes.com/sports/cricket/3
Navigated to page: https://timesofindia.indiatimes.com/sports/cricket/4
Navigated to page: https://timesofindia.indiatimes.com/sports/cricket/5
Navigated to page: https://timesofindia.indiatimes.com/sports/cricket/6
Navigated to page: https://timesofindia.indiatimes.com/sports/cricket/7
Navigated to page: https://timesofindia.indiatimes.com/sports/cricket/8
Navigated to page: https://timesofindia.indiatimes.com/sports/cricket/9
Navigated to page: https://timesofindia.indiatimes.com/sports/cricket/10
Navigated to page: https://timesofindia.indiatimes.com/sports/cricket/11
Navigated to page: https://timesofindia.indiatimes.com/sports/cricket/12
Navigated to page: https://timesofindia.indiatimes.com/sports/cricket/13
Navigated to page: https://timesofindia.indiatimes.com/sports/crick

In [8]:
print(f"Collected Cricket Hrefs: {len(cricket_hrefs)}")
print(f"Collected WWE Hrefs: {len(wwe_hrefs)}")
print(f"Collected Golf Hrefs: {len(golf_hrefs)}")
print(f"Collected Boxing Hrefs: {len(boxing_hrefs)}")
print(f"Collected Hockey Hrefs: {len(hockey_hrefs)}")
print(f"Collected NFL Hrefs: {len(nfl_hrefs)}")
print(f"Collected Football Hrefs: {len(football_hrefs)}")
print(f"Collected Tennis Hrefs: {len(tennis_hrefs)}")

Collected Cricket Hrefs: 586
Collected WWE Hrefs: 187
Collected Golf Hrefs: 5236
Collected Boxing Hrefs: 42
Collected Hockey Hrefs: 87
Collected NFL Hrefs: 1402
Collected Football Hrefs: 9436
Collected Tennis Hrefs: 1371


In [9]:
# Combine all the href lists into a single list
all_hrefs = (
    cricket_hrefs + 
    wwe_hrefs + 
    golf_hrefs + 
    boxing_hrefs + 
    hockey_hrefs + 
    nfl_hrefs + 
    tennis_hrefs + 
    football_hrefs
)

# Print the total number of collected hrefs
print(f"Total Collected Hrefs: {len(all_hrefs)}")

Total Collected Hrefs: 18347


In [10]:
hrefs_with_pairs = [[href, "NA"] for href in all_hrefs]

In [11]:
# Save the list to a file using pickle
with open('articles_from_sports.pkl', 'wb') as f:
    pickle.dump(hrefs_with_pairs, f)

print("Links saved to education_hrefs.pkl")

Links saved to education_hrefs.pkl


In [12]:
driver.quit()